In [8]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


CUDA available: True
Number of GPUs: 1
Current device: 0
Device name: NVIDIA GeForce RTX 4080 SUPER


In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch
from accelerate import infer_auto_device_map
import random

# Carica il modello dalla piattaforma Hugging Face
model_id = "stabilityai/stable-diffusion-xl-base-1.0"


# Configura la pipeline
pipeline = StableDiffusionXLPipeline.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    use_safetensors=True,  # Per performance migliori
).to("cuda")

Using Seed:  1049023134


Loading pipeline components...: 100%|██████████| 7/7 [00:02<00:00,  2.93it/s]


In [12]:
# Prompt di esempio
prompt = "A futuristic cityscape illuminated by neon lights at sunset, with towering skyscrapers made of glass and steel, flying vehicles in the sky, a bustling street market with diverse people, vibrant colors, hyper-realistic details, cinematic lighting, intricate reflections on the buildings, soft pink and orange hues in the sky, ultra-high definition, photorealistic, 16k resolution."
negative = "blurry, low quality, pixelated, out of focus, overexposed, underexposed, cartoonish, 2D art, poorly detailed, text artifacts, watermark, deformed buildings, unrealistic proportions, grainy textures, flat colors, monochrome, color bleeding, low resolution, bad composition."

num_inference_steps=50
guidance_scale=7.5

height = 1024  # Altezza in pixel
width = 1024  # Larghezza in pixel

# random seed
seed = random.randint(8, 2**32-1)
print("Using Seed: ",seed)
generator = torch.Generator(device="cpu").manual_seed(seed)

# Generazione dell'immagine
image = pipeline(
    prompt=prompt,
    negative_prompt=negative,                   # Prompt negativo
    height=height,
    width=width,
    num_inference_steps=num_inference_steps,    # Passi di inferenza (maggiore = più dettagli)
    guidance_scale= guidance_scale,              # Peso per il prompt (maggiore = risultati più aderenti)
    generator=generator
).images[0]

# Salva l'immagine generata
image.save("output_image.png")


Using Seed:  4285379855


100%|██████████| 50/50 [00:08<00:00,  6.24it/s]


To simulate such forms of image laundering and to avoid polarization, we follow the procedure used in the IEEE VIP Cup [13]. For each image of the test, a crop with random (large) size and position is selected, resized to 200 × 200 pixels, and compressed using a random JPEG quality factor from 65 to 100